In [ ]:
import os 
os.chdir("/home/sermengi/verifywise_dev/verifywise/BiasAndFairnessModule")
os.listdir()

In [ ]:
from src.config import ConfigManager
config_manager = ConfigManager()
model_config = config_manager.get_model_config()
model_config

In [ ]:
from src.data_loader import DataLoader
from src.model_loader import ModelLoader

In [ ]:
dataset_config = config_manager.get_dataset_config()
data_loader = DataLoader(dataset_config)
data_loader.load_data()

In [ ]:
sample_prompts = data_loader.get_sample_prompts([0, 1, 2, 3, 4])
sample_prompts

In [ ]:
model_loader = ModelLoader(
    model_id=model_config.huggingface.model_id,
    device=model_config.huggingface.device,
    max_new_tokens=model_config.huggingface.max_new_tokens,
    temperature=model_config.huggingface.temperature,
    top_p=model_config.huggingface.top_p,
    system_prompt=model_config.huggingface.system_prompt,
)

In [ ]:
model_loader.model_config.system_prompt

In [ ]:
formatted_prompt = model_loader._format_prompt(sample_prompts[0])
print(formatted_prompt)

In [ ]:
model_loader.predict(sample_prompts[0])

## Inference Module

In [ ]:
from src.config import ConfigManager
config_manager = ConfigManager()

In [ ]:
from src.inference import ModelInferencePipeline
import pandas as pd
model_inference_pipeline = ModelInferencePipeline()

In [ ]:
results = model_inference_pipeline.run_batch_inference()

In [ ]:
pd.read_csv("artifacts/inference_results.csv")

In [ ]:
preds = pd.Series(np.random.randint(0, 2, size=20)).map({0: "<=50K", 1: ">50K"})
answers = pd.Series(np.random.randint(0, 2, size=20)).map({0: "<=50K", 1: ">50K"})

In [ ]:
df = pd.DataFrame(results)

In [ ]:
df["answer"] = answers
df["pred"] = preds
df

In [ ]:
df.to_csv("data/adult-census-income-results.csv", index=False)

## Metric Implementations

In [ ]:
import numpy as np
import pandas as pd
from fairlearn.metrics import MetricFrame, false_positive_rate, true_positive_rate
from sklearn.metrics import precision_score, brier_score_loss

In [ ]:
y_true = np.array([1, 0, 1, 1, 0, 1, 0, 0, 1, 1])
y_pred = np.array([1, 0, 1, 1, 1, 1, 1, 0, 1, 0])
y_prob = np.clip(np.random.beta(2, 2, size=10), 0.01, 0.99)
sensitive_feature = pd.Series(["A", "A", "A", "A", "A", "B", "B", "B", "B", "B"])  # e.g., group A and B

* **Demographic parity**

In [ ]:
from fairlearn.metrics import demographic_parity_difference

In [ ]:
dp = demographic_parity_difference(y_true, y_pred, sensitive_features=sensitive_feature)
print(dp)

* **Equalized odds**

In [ ]:
from fairlearn.metrics import equalized_odds_difference

In [ ]:
eod = equalized_odds_difference(y_true, y_pred, sensitive_features=sensitive_feature, agg="worst_case")
print(eod)

* **Predictive Parity**

In [ ]:
def ppv(y_true, y_pred):
    return precision_score(y_true, y_pred, zero_division=0.0)

In [ ]:
metric_frame = MetricFrame(
    metrics=ppv,
    y_true=y_true,
    y_pred=y_pred,
    sensitive_features=sensitive_feature
)

In [ ]:
metric_frame.by_group

In [ ]:
metric_frame.difference()

* **Conditional Statistical Parity**

In [ ]:
y_pred = np.array([1, 0, 1, 1, 0, 1, 0, 1, 1, 0])
sensitive_feature = pd.Series(["M", "M", "M", "M", "M", "F", "F", "F", "F", "F"])
stratify_feature = pd.Series(["high", "low", "high", "low", "high", "low", "low", "high", "low", "high"])

In [ ]:
df = pd.DataFrame({
    "y_pred": y_pred,
    "sensitive": sensitive_feature,
    "stratify": stratify_feature
})

def conditional_statistical_parity(df, pred_col, sensitive_col, stratify_col):
    results = []
    for stratum in df[stratify_col].unique():
        subset = df[df[stratify_col] == stratum]

        # Dummy y_true (same shape as y_pred)
        dummy_y_true = np.zeros_like(subset[pred_col])

        metric_frame = MetricFrame(
            metrics=lambda y_true, y_pred: np.mean(y_pred),
            y_true=dummy_y_true,
            y_pred=subset[pred_col],
            sensitive_features=subset[sensitive_col]
        )
        group_rates = metric_frame.by_group
        disparity = group_rates.max() - group_rates.min()
        results.append((stratum, group_rates.to_dict(), disparity))
    return results

In [ ]:
csp_results = conditional_statistical_parity(df, "y_pred", "sensitive", "stratify")
for stratum, rates, disparity in csp_results:
    print(f"\nStratum: {stratum}")
    print("Group-wise selection rates:", rates)
    print("Disparity (max - min):", disparity)

* **Calibration**

In [ ]:
def calibration_metric(y_true, y_prob):
    return brier_score_loss(y_true, y_prob)

calibration = MetricFrame(
    metrics=calibration_metric,
    y_true=y_true,
    y_pred=y_prob,
    sensitive_features=sensitive_feature,
)

In [ ]:
calibration.by_group

In [ ]:
calibration.overall

* **Balance for positive class**

In [ ]:
def balance_for_positive_class(y_true, y_prob):
    return np.mean(y_prob[y_true == 1]) if np.any(y_true == 1) else np.nan

In [ ]:
bcp_metric = MetricFrame(
    metrics=balance_for_positive_class,
    y_true=y_true,
    y_pred=y_prob,
    sensitive_features=sensitive_feature
)

bcp_metric.by_group

In [ ]:
bcp_metric.overall

* **Balance for negative class**

In [ ]:
def balance_for_negative_class(y_true, y_prob):
    return np.mean(y_prob[y_true == 0]) if np.any(y_true == 0) else np.nan

bcn_metric = MetricFrame(
    metrics=balance_for_negative_class,
    y_true=y_true,
    y_pred=y_prob,
    sensitive_features=sensitive_feature
)

bcn_metric.by_group

In [ ]:
bcn_metric.overall

* **Predictive equality**

In [ ]:
metric_frame = MetricFrame(
    metrics=false_positive_rate,
    y_true=y_true,
    y_pred=y_pred,
    sensitive_features=sensitive_feature
)

metric_frame.by_group

In [ ]:
metric_frame.difference()

* **Conditional use accuracy equality**

In [ ]:
def negative_predictive_value(y_true, y_pred):
    return (sum((y_pred == 0) & (y_true == 0))) / (sum(y_pred == 0)) if np.any(y_pred == 0) else np.nan

metric_frame_npv = MetricFrame(
    metrics=negative_predictive_value,
    y_true=y_true,
    y_pred=y_pred,
    sensitive_features=sensitive_feature
)

metric_frame_ppv = MetricFrame(
    metrics=ppv,
    y_true=y_true,
    y_pred=y_pred,
    sensitive_features=sensitive_feature
)

print("PPV: ", metric_frame_ppv.by_group)
print("NPV: ", metric_frame_npv.by_group)

* **Equalized opportunities**

In [ ]:
metric_frame = MetricFrame(
    metrics=true_positive_rate,
    y_true=y_true,
    y_pred=y_pred,
    sensitive_features=sensitive_feature
)

metric_frame.by_group

## Post-Processing

In [ ]:
import os 
os.chdir("/home/sermengi/verifywise_dev/verifywise/BiasAndFairnessModule")
os.listdir()

In [ ]:
import pandas as pd

In [ ]:
from src.config import ConfigManager
from src.postprocessing import PostProcessor

config_manager = ConfigManager()
post_processor = PostProcessor(config_manager)

In [ ]:
post_processor.run()

In [ ]:
pd.read_csv("artifacts/postprocessed_results.csv")

In [ ]:
post_processor.df

In [ ]:
post_processor.expand_protected_attributes()
post_processor.df

In [ ]:
post_processor.encode_binary_columns()
post_processor.encode_protected_attributes()
post_processor.df

## Metric Registry

In [ ]:
import os 
os.chdir("/home/sermengi/verifywise_dev/verifywise/BiasAndFairnessModule")
os.listdir()

In [ ]:
from src.metric_registry import METRIC_REGISTRY, get_metric, list_metrics
from src.metrics import *

In [ ]:
list_metrics()

In [ ]:
equal_selection_parity = get_metric("equal_selection_parity")
equal_selection_parity

## Evaluator

In [ ]:
import os 
os.chdir("/home/sermengi/verifywise_dev/verifywise/BiasAndFairnessModule")
os.listdir()

In [ ]:
from src.evaluator import FairnessEvaluator
from src.config import ConfigManager

config_manager = ConfigManager()
metrics_config = config_manager.get_metrics_config()
results_path = "data/adult-census-income-results.csv"

evaluator = FairnessEvaluator(
    config_manager=config_manager,
)

In [ ]:
evaluator.config.fairness.metrics

In [ ]:
metric_functions = evaluator.evaluate()

In [ ]:
metric_functions["fairness"]

## Example

In [ ]:
import os 
os.chdir("/home/sermengi/verifywise_dev/verifywise/BiasAndFairnessModule")
os.listdir()

In [ ]:
from src.config import ConfigManager

config_manager = ConfigManager()
artifacts_config = config_manager.get_artifacts_config()
artifacts_config

In [ ]:
artifacts_config.postprocessed_results_path

## Metric Tests

In [ ]:
import pandas as pd

In [ ]:
import os 
os.chdir("/home/sermengi/verifywise_dev/verifywise/BiasAndFairnessModule")
os.listdir()

In [ ]:
from src.metrics import *
from src.metric_registry import METRIC_REGISTRY, list_metrics, get_metric
from fairlearn.metrics import selection_rate_difference

In [ ]:
get_metric("demographic_parity")(y_true, y_pred, protected_attributes=sensitive_feature)

In [ ]:
METRIC_REGISTRY.get("demographic_parity")(y_true, y_pred, sensitive_feature.map({"A":0, "B":1}))

In [ ]:
"""Set up test data."""
# Create sample data for testing
np.random.seed(42)
n_samples = 1000

# Create synthetic data with bias
sensitive_attr = np.random.choice([0, 1], size=n_samples, p=[0.7, 0.3])
y_true = np.random.choice([0, 1], size=n_samples, p=[0.6, 0.4])

# Create biased predictions (group 1 has higher positive rate)
y_pred = np.zeros(n_samples)
for i in range(n_samples):
    if sensitive_attr[i] == 0:
        y_pred[i] = np.random.choice([0, 1], p=[0.8, 0.2])
    else:
        y_pred[i] = np.random.choice([0, 1], p=[0.6, 0.4])

# Create continuous scores
y_scores = np.random.random(n_samples)
y_scores[sensitive_attr == 1] += 0.2

In [ ]:
get_metric("demographic_parity")(y_true, y_pred, sensitive_attr)

In [ ]:
def test_calculate_demographic_parity():
    """Test demographic parity calculation using Fairlearn."""
    demographic_parity = get_metric("demographic_parity")
    
    # Test with biased data
    result = demographic_parity(y_true, y_pred, )
    
    # Should detect bias (group 1 has higher positive rate)
    self.assertGreater(result, 0.0)
    self.assertLessEqual(result, 1.0)
    
    # Test with equal predictions
    equal_pred = np.ones(self.n_samples)
    result_equal = engine.calculate_demographic_parity(equal_pred, self.sensitive_attr)
    self.assertEqual(result_equal, 0.0)

In [ ]:
y_pred = (y_scores > 0.5).astype(int)
selection_rate_difference(y_pred, y_pred, sensitive_features=sensitive_attr)

In [ ]:
bpc = get_metric("balance_positive_class")(y_true, y_scores, protected_attributes=sensitive_attr)

In [ ]:
bpc.overall

## Unit Tests

In [35]:
import os 
os.chdir("/home/sermengi/verifywise_dev/verifywise/BiasAndFairnessModule")
os.listdir()

['.venv',
 'src',
 '.gitignore',
 'main.py',
 'tests',
 'configs',
 'README.md',
 'pyproject.toml',
 'y_pred_no_seed.npy',
 '.python-version',
 'uv.lock',
 'notebooks',
 'artifacts',
 'Makefile',
 '.mypy_cache',
 'requirements.txt',
 '.pytest_cache',
 '.env']

In [36]:
from src.metrics import *

In [37]:
from tests.base_test_metrics import BaseMetricsTestCase
base_metrics_test_case = BaseMetricsTestCase()
base_metrics_test_case.setUp()

In [38]:
y_true = base_metrics_test_case.y_true
y_pred = base_metrics_test_case.y_pred
y_scores = base_metrics_test_case.y_scores
y_protected_attr = base_metrics_test_case.protected_attributes
y_legitimate_attr = base_metrics_test_case.legitimate_attributes

In [43]:
bnc = balance_negative_class(y_true=y_true, y_pred_proba=y_scores, protected_attributes=y_protected_attr)

In [44]:
bnc.ratio()

np.float64(0.6840727903151731)

In [45]:
bnc.difference()

np.float64(0.18545292723894335)